In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
from PIL import Image
from tensorflow.keras.callbacks import TensorBoard


# Preprocess Data

### Read Images

In [46]:

new_width  = 512
new_height = 512
img = Image.open('Ala_Idris\Ala_Idris (1).png') # image extension *.png,*.jpg
img = img.resize((new_width, new_height), Image.ANTIALIAS)
img = img.convert('RGB')
img = np.array(img)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
data = [img]
label = np.array([0])




for i in range(2,101):
    img = Image.open('Ala_Idris\Ala_Idris (1).png') # image extension *.png,*.jpg
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.resize((new_width, new_height), Image.ANTIALIAS)
    img = img.convert('RGB')
    img = np.array(img)
    data = np.append(data, [img], axis=0)
    label = np.append(label, [0])

for i in range(1,101):
    img = Image.open('Ala_Idris\Ala_Idris (1).png') # image extension *.png,*.jpg
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.resize((new_width, new_height), Image.ANTIALIAS)
    img = img.convert('RGB')
    img = np.array(img)
    data = np.append(data, [img], axis=0)
    label = np.append(label, [1])

for i in range(1,101):
    img = Image.open('Ala_Idris\Ala_Idris (1).png') # image extension *.png,*.jpg
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.resize((new_width, new_height), Image.ANTIALIAS)
    img = img.convert('RGB')
    img = np.array(img)
    data = np.append(data, [img], axis=0)
    label = np.append(label, [2])

for i in range(1,101):
    img = Image.open('Ala_Idris\Ala_Idris (1).png') # image extension *.png,*.jpg
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.resize((new_width, new_height), Image.ANTIALIAS)
    img = img.convert('RGB')
    img = np.array(img)
    data = np.append(data, [img], axis=0)
    label = np.append(label, [3])

data = data / 255


In [47]:
data.shape

(400, 512, 512, 3)

### Split Data

In [48]:
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

## Data argumetaions

In [49]:
X_train_arg = X_train.astype(int)
y_train_arg = y_train

#### Horizontal Shift

In [7]:

# datagen = ImageDataGenerator(width_shift_range=[-100,100])
# it = datagen.flow(X_train, batch_size=320,shuffle = False)
# for i in range(1):
# 	batch = it.next()	
# 	# image = batch[1].astype('uint8')
# 	X_train_arg = np.append(X_train_arg, batch.astype(int), axis=0)
# 	y_train_arg = np.append(y_train_arg, y_train, axis=0)




#### Vertical shift

In [8]:
# datagen = ImageDataGenerator(height_shift_range=0.5)
# it = datagen.flow(X_train, batch_size=320,shuffle = False)
# for i in range(1):
# 	batch = it.next()	
# 	# image = batch[1].astype('uint8')
# 	X_train_arg = np.append(X_train_arg, batch.astype(int), axis=0)
# 	y_train_arg = np.append(y_train_arg, y_train, axis=0)



#### 

#### Rotation

In [9]:
# datagen = ImageDataGenerator(rotation_range=90)
# it = datagen.flow(X_train, batch_size=320,shuffle = False)
# for i in range(1):
# 	batch = it.next()	
# 	# image = batch[1].astype('uint8')
# 	X_train_arg = np.append(X_train_arg, batch.astype(int), axis=0)
# 	y_train_arg = np.append(y_train_arg, y_train, axis=0)



## Split Validation

In [50]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_arg, y_train_arg, test_size=0.2, random_state=42)

In [51]:
X_train.shape

(256, 512, 512, 3)

## Auto Encoding

In [60]:
input_img = keras.Input(shape=(512, 512, 3))

x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)


x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu',padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)

decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# autoencoder.summary()


autoencoder.fit(X_train, X_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(X_valid, X_valid),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

encoder = keras.Model(input_img, encoded)
encoded_imgs = encoder.predict(X_test)


Epoch 1/50


ResourceExhaustedError:  OOM when allocating tensor with shape[128,512,512,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node model_11/conv2d_78/Relu (defined at C:\Users\rayanf\AppData\Local\Temp\ipykernel_1720\254247728.py:27) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_4496]

Function call stack:
train_function


# Define Network Based on ResNet50

In [13]:
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
  return output_ims

In [14]:
X_train = preprocess_image_input(X_train)
X_valid = preprocess_image_input(X_valid)


In [15]:
def feature_extractor(inputs):
  feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(511, 511,3),
                                              include_top=False,
                                              weights='imagenet')(inputs)
  return feature_extractor

In [16]:
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    # x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(4, activation="softmax", name="classification")(x)
    return x

In [21]:
def final_model(inputs):
    # resize = tf.keras.layers.UpSampling2D(size=(7,7))(inputs)
    resize = inputs
    # resnet_feature_extractor = feature_extractor(resize)
    # classification_output = classifier(resnet_feature_extractor)
    classification_output = classifier(resize)
    return classification_output

In [18]:
def compile_model():
    inputs = tf.keras.layers.Input(shape=(511,511,3))
    classification_output = final_model(inputs) 
    model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
    model.compile(optimizer='SGD', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
    return model

In [22]:
model = compile_model()
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 511, 511, 3)]     0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 3)                 0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               2048      
_________________________________________________________________
classification (Dense)       (None, 4)                 2052      
Total params: 4,100
Trainable params: 4,100
Non-trainable params: 0
_________________________________________________________________


In [24]:
EPOCHS = 4
y_train = y_train.reshape(256,1).astype('uint8')
y_valid = y_valid.reshape(64,1).astype('uint8')
history = model.fit(X_train, y_train, epochs=EPOCHS, validation_data = (X_valid, y_valid), batch_size=8)
# X_train.shape
# y_valid.shape
# print(y_train.shape)
# print(X_valid.shape)
# y_train


Epoch 1/4
32/32 [==============================] - 1s 29ms/step - loss: 1.3859 - accuracy: 0.2617 - val_loss: 1.3886 - val_accuracy: 0.1719
Epoch 2/4
32/32 [==============================] - 1s 19ms/step - loss: 1.3854 - accuracy: 0.2383 - val_loss: 1.3898 - val_accuracy: 0.1719
Epoch 3/4
32/32 [==============================] - 1s 37ms/step - loss: 1.3850 - accuracy: 0.2773 - val_loss: 1.3908 - val_accuracy: 0.1719
Epoch 4/4
32/32 [==============================] - 1s 17ms/step - loss: 1.3846 - accuracy: 0.2773 - val_loss: 1.3918 - val_accuracy: 0.1719
